In [1]:
import pyprind
import pandas as pd
import os
import sys
import numpy
basepath = 'aclImdb'

labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000, stream=sys.stdout)
df = pd.DataFrame()
for s in ('test', 'train'):
   for l in ('pos', 'neg'):
     path = os.path.join(basepath, s, l)
     for file in sorted(os.listdir(path)):
        with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
          txt = infile.read()
        df = df._append([[txt, labels[l]]], ignore_index=True)
        pbar.update()
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:10


In [2]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [3]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.rename(columns={'0': 'review', '1': 'sentiment'}, inplace=True)
df.head(3)
df.shape

(50000, 2)

In [4]:
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [5]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [6]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [7]:
import re
def preprocessor(text):
    text = re.sub('<[^]*', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) +' '.join(emoticons).replace('-', ''))
    return text

In [8]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [9]:
def tokenizer(text):
    return text.split()

In [11]:
from nltk.stem.porter import PorterStemmer

In [12]:
porter = PorterStemmer()

In [13]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/musoye/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None, lowercase=False, preprocessor=None)
small_param_grid = [
 {
  'vect__ngram_range': [(1, 1)],
  'vect__stop_words': [None],
  'vect__tokenizer': [tokenizer, tokenizer_porter],
  'clf__penalty': ['l2'],
  'clf__C': [1.0, 10.0]
  },
  {
   'vect__ngram_range': [(1, 1)],
   'vect__stop_words': [stop, None],
   'vect__tokenizer': [tokenizer],
   'vect__use_idf':[False],
   'vect__norm':[None],
   'clf__penalty': ['l2'],
   'clf__C': [1.0, 10.0]
   },
]
lr_tfidf = Pipeline([('vect', tfidf),('clf', LogisticRegression(solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid, scoring='accuracy', cv=5, verbose=2, n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/home/musoye/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f5bcfbc0280>; total time=   7.7s


/home/musoye/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=1.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f5bcfbc0280>; total time=   7.6s


...

[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f5bcfbc0280>, vect__use_idf=False; total time=  33.4s


/home/musoye/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END clf__C=10.0, clf__penalty=l2, vect__ngram_range=(1, 1), vect__norm=None, vect__stop_words=None, vect__tokenizer=<function tokenizer at 0x7f5bcfbc0280>, vect__use_idf=False; total time=  26.0s


/home/musoye/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x7f5bcfbc0280>,
                                              <function tokenizer_porter at 0x7f5bae99e050>...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x7f5bcfbc0280>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=2)

In [20]:
gs_lr_tfidf.best_score_

0.8869647110577885